In [1]:
from openai_chat_agent.utils.google_search import GoogleSearch


In [3]:
query = "What is the typical life span of a cat?"
search = GoogleSearch()
results = search(query)
print(results)
print(f'{len(results.split())} words.')


I enquired:
"What is the typical life span of a cat?"

I learned:
The average lifespan of domestic cats has increased over the years. In the 1980s, the average lifespan was seven years, which increased to nine years in 1995, and is now around 15 years in 2021. However, reliable information on cat lifespans is limited. Studies have estimated the mean lifespan to be between 13 and 20 years, with an average of around 15 years. Mixed-breed cats tend to have a longer lifespan compared to purebred cats, and breed-specific lifespans can vary. For example, Maine Coon cats have an average lifespan of 10-13 years, while Siamese cats can live to be 15-20 years old.

My references:
Aging in cats - Wikipedia: https://en.wikipedia.org/wiki/Aging_in_cats
How Long Do Cats Live? | PetMD: https://www.petmd.com/cat/care/how-long-do-cats-live
What Is the Average Lifespan of the Common Cat?: https://www.thesprucepets.com/lifespan-of-cats-552035

141 words.


In [4]:
query = "How can I learn more about prompt engineering?"
search = GoogleSearch()
results = search(query)
print(results)
print(f'{len(results.split())} words.')

I enquired:
"How can I learn more about prompt engineering?"

I learned:
The information provided includes details about a free course on prompt engineering, its importance and challenges, and a review of the Learn Prompting course. It also explains what prompt engineering is and how it can improve communication with AI tools. The course offers comprehensive educational resources, including links to articles, blogs, practical examples, and tasks related to prompt engineering. It provides a non-linear learning model, allowing learners to explore topics of interest. The course aims to provide a comprehensive overview and hands-on experience in prompt engineering, catering to learners of all levels.

My references:
Learn Prompting 101: Prompt Engineering Course & Challenges ...: https://towardsai.net/p/machine-learning/learn-prompting-101-prompt-engineering-course
Want to learn more about prompt engineering? This free course can ...: https://www.zdnet.com/article/want-to-learn-more-about-